In [60]:
import pandas as pd
import numpy as np

In [61]:
path = "/home/taha/Documents/ML/iris.txt"
df = pd.read_csv(path)

train = df.sample(frac=0.8,random_state=200)
test = df.drop(train.index)

training_data = train.values.tolist()
testing_data = test.values.tolist()

FileNotFoundError: File b'/home/taha/Documents/ML/iris.txt' does not exist

In [64]:
from sklearn import datasets
import pandas as pd

data = datasets.load_iris().data
target = datasets.load_iris().target
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [70]:
data = pd.concat([pd.DataFrame(data),pd.DataFrame(target)],axis=1)

In [71]:
data.columns = datasets.load_iris()['feature_names'] + ['Species']

In [72]:
train=data.sample(frac=0.8)
test=data.drop(train.index)

In [74]:
# Column labels.
header = ["sepal length", "sepal width", "petal length", "petal width" , "Species"]

In [67]:
def unique_vals(rows,col):
    return set([row[col] for row in rows])

In [68]:
def class_counts(rows):
    counts = dict()
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [9]:
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

In [73]:
header

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'Species'],
      dtype='object')

In [10]:
class Question:
    
    def __init__(self,column,value):
        self.column = column
        self.value = value
    
    def match(self, example):
        value = example[self.column]
        if is_numeric(value):
            return value >= self.value
        else:
            return value == self.value
    
    
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        
        return "Is %s %s %s?" %(
            header[self.column], condition, str(self.value))
        
    

In [11]:
def partition(rows,question):
    
    true_rows, false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
            
    return true_rows,false_rows       
        

In [12]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        lbl_prob = counts[lbl]/float(len(rows))
        impurity -= lbl_prob**2
    return impurity    

In [13]:
def information_gain(left,right,current_uncertainity):
    p = float(len(left))/float(len(left)+len(right))
    return current_uncertainity - p * gini(left) - (1-p) * gini(right)

In [14]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        
        for val in values:
            
            question = Question(col,val)
            true_rows,false_rows = partition(rows,question)
            
            if len(true_rows) == 0 or len(false_rows)==0:
                continue
            
            gain = information_gain(true_rows,false_rows,current_uncertainity)
            
            if gain >= best_gain:
                best_gain,best_question = gain,question
                
    return best_gain,best_question

In [15]:
class Leaf:
    
    def __init__(self,rows):
        self.predictions = class_counts(rows)

In [16]:
class Decision_Node:
    
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        

In [17]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    
    if gain==0:
        return Leaf(rows)
    
    true_rows,false_rows = partition(rows,question)
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question,true_branch,false_branch)
    

In [18]:
def print_tree(node,spacing=""):
    
    if isinstance(node,Leaf):
        print(spacing + "Predict" + str(node.predictions))
        return
        
    print(spacing + str(node.question))
    
    print(spacing + "True-->:")
    print_tree(node.true_branch,spacing + " ")
    
    print(spacing + "False-->:")
    print_tree(node.false_branch,spacing + " ")
    

In [19]:
def classify(row,node):
    
    if isinstance(node,Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row,node.true_branch)
    
    else:
        return classify(row,node.false_branch)
    

In [20]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total*100)) + "%"
        
    return probs    

In [21]:
my_tree = build_tree(training_data)

In [22]:
print_tree(my_tree)

Is petal width >= 1.0?
True-->:
 Is petal length >= 4.8?
 True-->:
  Is petal width >= 1.8?
  True-->:
   Is petal length >= 4.9?
   True-->:
    Predict{'Iris-virginica': 33}
   False-->:
    Is sepal width >= 3.2?
    True-->:
     Predict{'Iris-versicolor': 1}
    False-->:
     Predict{'Iris-virginica': 2}
  False-->:
   Is petal length >= 5.0?
   True-->:
    Is petal width >= 1.6?
    True-->:
     Is petal length >= 5.8?
     True-->:
      Predict{'Iris-virginica': 1}
     False-->:
      Predict{'Iris-versicolor': 1}
    False-->:
     Predict{'Iris-virginica': 3}
   False-->:
    Predict{'Iris-versicolor': 2}
 False-->:
  Predict{'Iris-versicolor': 36}
False-->:
 Predict{'Iris-setosa': 40}


In [23]:
#Testing
count=0
for row in testing_data:
    
    predicted = print_leaf(classify(row, my_tree))
    
    if row[-1] in predicted:
        count +=1 
    
    print("Actual Label: %s Predicted Label: %s" %
           (row[-1],predicted))

print("Accuracy = %s %%" %(count/len(testing_data)*100))    

Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-setosa Predicted Label: {'Iris-setosa': '100%'}
Actual Label: Iris-versicolor Predicted Label: {'Iris-versicolor': '100%'}
Actual Label: Iris-versicolor Predicted Label: {'Iris-versicolor': '100%'}
Actual Label: Iris-versicolor Predicted Label: {'Iris-versicolor': '100%'}
Actual Label: Iris-versicolor Predicted Label: {'Iris-versicolor': '100%'}
Actual Label: Iris-versicolor Predicted Label: {'Iris-versicolor': '100%'}
Actual Label: Iris-ver